In [1]:
!python -V

Python 3.9.7


In [1]:
!ls data

green_tripdata_2021-01.csv
green_tripdata_2021-02.csv


In [2]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [4]:
df = pd.read_csv('./data/green_tripdata_2021-01.csv')

C:\Users\alexe\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [6]:
def load_prepare_dataframe(filename):
    df = pd.read_csv(filename)
    
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime

    df.duration = df.duration.apply(lambda dt: dt.total_seconds() / 60)

    df = df[df.trip_distance > 0]
    df = df[df.trip_type == 2]

    df = df[df.duration > 0]
    df = df[df.duration <= 60]

    df = df.reset_index(drop=True)
    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = load_prepare_dataframe('./data/green_tripdata_2021-01.csv')
df_val = load_prepare_dataframe('./data/green_tripdata_2021-02.csv')

C:\Users\alexe\AppData\Local\Temp/ipykernel_3836/1694870714.py:1: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  df_train = load_prepare_dataframe('./data/green_tripdata_2021-01.csv')


In [8]:
dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [11]:
y_pred = lr.predict(X_train)
y_pred[y_pred < 1] = 1 
mean_squared_error(y_train, y_pred, squared=False)

4.6163373320088805

In [12]:
y_pred = lr.predict(X_val)
y_pred[y_pred < 1] = 1 
mean_squared_error(y_val, y_pred, squared=False)

8.080354199560718

In [109]:
lr = Lasso(alpha=0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
y_pred[y_pred < 1] = 1 
mean_squared_error(y_val, y_pred, squared=False)

7.580612148740968

In [106]:
import pickle

In [107]:
with open('model.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [85]:
df_train['PU_DO'] = df_train.PULocationID + '_' + df_train.DOLocationID
df_val['PU_DO'] = df_val.PULocationID + '_' + df_val.DOLocationID

In [89]:
categorical = ['PU_DO']
numerical = ['trip_distance']

In [90]:
dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [96]:
lr = Lasso(alpha=0.001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
y_pred[y_pred < 1] = 1 
mean_squared_error(y_val, y_pred, squared=False)

8.141342304753202